In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
import matplotlib
matplotlib.use('Agg')  # 'Agg' is a non-interactive backend suitable for scripts
import matplotlib.pyplot as plt

In [2]:
all_xray_df = pd.read_csv('E:/A__CVPR/Dataset/bbox/Data_Entry_2017_v2020.csv')
all_image_paths = {os.path.basename(x): x for x in 
                glob(os.path.join('D:/New CX/CXR8/images', '*.png'))}
print('Scans found:', len(all_image_paths), ', Total Headers', all_xray_df.shape[0])
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)


#all_xray_df['Patient Age'] = all_xray_df['Patient Age'].map(lambda x: int(x[:-1]))
all_xray_df.sample(3)

Scans found: 112120 , Total Headers 112120


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],path
40425,00010530_021.png,No Finding,20,10530,37,F,AP,2500,2048,0.168,0.168,D:/New CX/CXR8/images\00010530_021.png
78579,00019271_062.png,No Finding,62,19271,54,M,AP,2500,2048,0.168,0.168,D:/New CX/CXR8/images\00019271_062.png
76901,00018881_000.png,No Finding,0,18881,58,M,PA,2992,2769,0.143,0.143,D:/New CX/CXR8/images\00018881_000.png


In [3]:
label_counts = all_xray_df['Finding Labels'].value_counts()[:15]
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [4]:
all_xray_df['Finding Labels'] = all_xray_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
from itertools import chain
all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        all_xray_df[c_label] = all_xray_df['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)
all_xray_df.sample(3)

All Labels (14): ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
34769,00009168_003.png,,3,9168,43,F,AP,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9485,00002464_000.png,Emphysema,0,2464,52,F,AP,2500,2048,0.168,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41857,00010790_033.png,,33,10790,55,F,AP,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
all_labels

['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Effusion',
 'Emphysema',
 'Fibrosis',
 'Hernia',
 'Infiltration',
 'Mass',
 'Nodule',
 'Pleural_Thickening',
 'Pneumonia',
 'Pneumothorax']

In [6]:
# keep at least 1000 cases
MIN_CASES = 1000
all_labels = [c_label for c_label in all_labels if all_xray_df[c_label].sum()>MIN_CASES]
print('Clean Labels ({})'.format(len(all_labels)), 
      [(c_label,int(all_xray_df[c_label].sum())) for c_label in all_labels])

Clean Labels (13) [('Atelectasis', 11559), ('Cardiomegaly', 2776), ('Consolidation', 4667), ('Edema', 2303), ('Effusion', 13317), ('Emphysema', 2516), ('Fibrosis', 1686), ('Infiltration', 19894), ('Mass', 5782), ('Nodule', 6331), ('Pleural_Thickening', 3385), ('Pneumonia', 1431), ('Pneumothorax', 5302)]


In [7]:
# since the dataset is very unbiased, we can resample it to be a more reasonable collection
# weight is 0.1 + number of findings
sample_weights = all_xray_df['Finding Labels'].map(lambda x: len(x.split('|')) if len(x)>0 else 0).values + 4e-2
sample_weights /= sample_weights.sum()
all_xray_df = all_xray_df.sample(40000, weights=sample_weights)

label_counts = all_xray_df['Finding Labels'].value_counts()[:15]
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [8]:
label_counts = 100*np.mean(all_xray_df[all_labels].values,0)
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
ax1.set_xticklabels(all_labels, rotation = 90)
ax1.set_title('Adjusted Frequency of Diseases in Patient Group')
_ = ax1.set_ylabel('Frequency (%)')

In [9]:
all_xray_df['disease_vec'] = all_xray_df.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])

In [10]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(all_xray_df, 
                                  test_size = 0.25, 
                                  random_state = 2018,
                                  stratify = all_xray_df['Finding Labels'].map(lambda x: x[:4]))
print('train', train_df.shape[0], 'validation', valid_df.shape[0])

train 30000 validation 10000


In [11]:
valid_df['newLabel'] = valid_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)
train_df['newLabel'] = train_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (128, 128)
core_idg = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip=True, 
                              vertical_flip=False, 
                              height_shift_range=0.05, 
                              width_shift_range=0.1, 
                              rotation_range=5, 
                              shear_range=0.1,
                              fill_mode='reflect',
                              zoom_range=0.15)


In [13]:
train_gen = core_idg.flow_from_dataframe(dataframe=train_df, 
                            directory=None,
                            x_col = 'path',
                            y_col = 'newLabel', 
                            class_mode = 'categorical',
                            classes = all_labels,
                            target_size = IMG_SIZE,
                            color_mode = 'grayscale',
                            batch_size = 32)

valid_gen = core_idg.flow_from_dataframe(dataframe=valid_df, 
                            directory=None,
                            x_col = 'path',
                            y_col = 'newLabel', 
                            class_mode = 'categorical',
                            classes = all_labels,
                            target_size = IMG_SIZE,
                            color_mode = 'grayscale',
                            batch_size = 256) # we can use much larger batches for evaluation

test_X, test_Y = next(core_idg.flow_from_dataframe(dataframe=valid_df, 
                            directory=None,
                            x_col = 'path',
                            y_col = 'newLabel', 
                            class_mode = 'categorical',
                            classes = all_labels,
                            target_size = IMG_SIZE,
                            color_mode = 'grayscale',
                            batch_size = 1024))

Found 28280 validated image filenames belonging to 13 classes.
Found 9432 validated image filenames belonging to 13 classes.
Found 9432 validated image filenames belonging to 13 classes.


In [14]:
t_x, t_y = next(train_gen)
fig, m_axs = plt.subplots(4, 4, figsize = (16, 16))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone', vmin = -1.5, vmax = 1.5)
    c_ax.set_title(', '.join([n_class for n_class, n_score in zip(all_labels, c_y) 
                            if n_score>0.5]))
    c_ax.axis('off')

In [15]:
from keras.applications.mobilenet import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.models import Sequential
from keras import optimizers, callbacks, regularizers
base_mobilenet_model = MobileNet(input_shape =  t_x.shape[1:], 
                                include_top = False, weights = None)
multi_disease_model = Sequential()
multi_disease_model.add(base_mobilenet_model)
multi_disease_model.add(GlobalAveragePooling2D())
multi_disease_model.add(Dropout(0.5))
multi_disease_model.add(Dense(512))
multi_disease_model.add(Dropout(0.5))
multi_disease_model.add(Dense(len(all_labels), activation = 'sigmoid'))
multi_disease_model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                          metrics = ['binary_accuracy', 'mae'])
multi_disease_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_128 (Functio  (None, 4, 4, 1024)       3228288   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 13)                6

In [16]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

# Modify the weight path to end with `.weights.h5`
weight_path = "{}_weights.best.weights.h5".format('xray_class')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                            save_best_only=True, mode='min', save_weights_only=True)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=5)

callbacks_list = [checkpoint, early]


In [17]:
multi_disease_model.fit(train_gen, 
                        steps_per_epoch=100,
                        validation_data=(test_X, test_Y), 
                        epochs=1, 
                        callbacks=callbacks_list)


100/100 [==============================] - ETA: 0s - loss: 0.4651 - binary_accuracy: 0.8415 - mae: 0.2094
Epoch 1: val_loss improved from inf to 0.36253, saving model to xray_class_weights.best.weights.h5
100/100 [==============================] - 68s 548ms/step - loss: 0.4651 - binary_accuracy: 0.8415 - mae: 0.2094 - val_loss: 0.3625 - val_binary_accuracy: 0.8673 - val_mae: 0.2446


In [18]:
for c_label, s_count in zip(all_labels, 100*np.mean(test_Y,0)):
    print('%s: %2.2f%%' % (c_label, s_count))

Atelectasis: 25.00%
Cardiomegaly: 5.27%
Consolidation: 11.91%
Edema: 4.98%
Effusion: 30.27%
Emphysema: 4.69%
Fibrosis: 3.12%
Infiltration: 40.04%
Mass: 11.82%
Nodule: 14.16%
Pleural_Thickening: 6.45%
Pneumonia: 1.76%
Pneumothorax: 12.99%


In [19]:
pred_Y = multi_disease_model.predict(test_X, batch_size = 32, verbose = True)

32/32 [==============================] - 1s 14ms/step


In [20]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fig, c_ax = plt.subplots(1, 1, figsize=(9, 9))

# Ensure all_labels is defined and contains the class labels
for (idx, c_label) in enumerate(all_labels):
    # Ensure test_Y and pred_Y are correctly shaped and formatted
    fpr, tpr, thresholds = roc_curve(test_Y[:, idx].astype(int), pred_Y[:, idx])
    c_ax.plot(fpr, tpr, label='%s (AUC:%0.2f)' % (c_label, auc(fpr, tpr)))

c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')

# Save and display the plot
fig.savefig('barely_trained_net.png')
plt.show()


C:\Users\Mohammad Sheakh\AppData\Local\Temp\ipykernel_11164\3785646183.py:18: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [21]:
optimizers_list = [('sgd', optimizers.SGD(learning_rate=0.1)),
                  ('sgd_momentum', optimizers.SGD(learning_rate=0.1, momentum=0.9)),
                  ('adagrad', optimizers.Adagrad()),
                  ('adadelta', optimizers.Adadelta()),
                  ('adam', optimizers.Adam())]

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=5)
callbacks_list = [early]


In [22]:
multi_disease_model.fit_generator(train_gen, 
                                steps_per_epoch = 100,
                                validation_data =  (test_X, test_Y), 
                                epochs = 100, 
                                callbacks = callbacks_list)

C:\Users\Mohammad Sheakh\AppData\Local\Temp\ipykernel_11164\1089579743.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  multi_disease_model.fit_generator(train_gen,


Epoch 1/100
100/100 [==============================] - 48s 477ms/step - loss: 0.3834 - binary_accuracy: 0.8580 - mae: 0.2081 - val_loss: 0.3926 - val_binary_accuracy: 0.8673 - val_mae: 0.1636
Epoch 2/100
100/100 [==============================] - 46s 456ms/step - loss: 0.3606 - binary_accuracy: 0.8649 - mae: 0.2053 - val_loss: 0.4356 - val_binary_accuracy: 0.8673 - val_mae: 0.1503
Epoch 3/100
100/100 [==============================] - 45s 446ms/step - loss: 0.3604 - binary_accuracy: 0.8622 - mae: 0.2090 - val_loss: 0.4421 - val_binary_accuracy: 0.8673 - val_mae: 0.1536
Epoch 4/100
100/100 [==============================] - 43s 433ms/step - loss: 0.3536 - binary_accuracy: 0.8639 - mae: 0.2076 - val_loss: 0.4320 - val_binary_accuracy: 0.8673 - val_mae: 0.1524
Epoch 5/100
100/100 [==============================] - 42s 422ms/step - loss: 0.3455 - binary_accuracy: 0.8657 - mae: 0.2038 - val_loss: 0.3851 - val_binary_accuracy: 0.8673 - val_mae: 0.1637
Epoch 6/100
100/100 [===================

In [1]:
# load the best weights
multi_disease_model.load_weights(weight_path)

NameError: name 'multi_disease_model' is not defined

In [24]:
pred_Y = multi_disease_model.predict(test_X, batch_size = 32, verbose = True)

32/32 [==============================] - 0s 14ms/step


In [25]:
# look at how often the algorithm predicts certain diagnoses 
for c_label, p_count, t_count in zip(all_labels, 
                                    100*np.mean(pred_Y,0), 
                                    100*np.mean(test_Y,0)):
    print('%s: Dx: %2.2f%%, PDx: %2.2f%%' % (c_label, t_count, p_count))

Atelectasis: Dx: 25.00%, PDx: 35.50%
Cardiomegaly: Dx: 5.27%, PDx: 8.53%
Consolidation: Dx: 11.91%, PDx: 9.94%
Edema: Dx: 4.98%, PDx: 16.93%
Effusion: Dx: 30.27%, PDx: 32.91%
Emphysema: Dx: 4.69%, PDx: 8.52%
Fibrosis: Dx: 3.12%, PDx: 10.40%
Infiltration: Dx: 40.04%, PDx: 30.81%
Mass: Dx: 11.82%, PDx: 15.05%
Nodule: Dx: 14.16%, PDx: 18.83%
Pleural_Thickening: Dx: 6.45%, PDx: 13.08%
Pneumonia: Dx: 1.76%, PDx: 10.57%
Pneumothorax: Dx: 12.99%, PDx: 19.19%


In [2]:
from sklearn.metrics import roc_curve, auc
fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for (idx, c_label) in enumerate(all_labels):
    fpr, tpr, thresholds = roc_curve(test_Y[:,idx].astype(int), pred_Y[:,idx])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('trained_net.png')

NameError: name 'plt' is not defined

In [27]:
sickest_idx = np.argsort(np.sum(test_Y, 1)<1)
fig, m_axs = plt.subplots(4, 2, figsize = (16, 32))
for (idx, c_ax) in zip(sickest_idx, m_axs.flatten()):
    c_ax.imshow(test_X[idx, :,:,0], cmap = 'bone')
    stat_str = [n_class[:6] for n_class, n_score in zip(all_labels, 
                                                                  test_Y[idx]) 
                            if n_score>0.5]
    pred_str = ['%s:%2.0f%%' % (n_class[:4], p_score*100)  for n_class, n_score, p_score in zip(all_labels, 
                                                                  test_Y[idx], pred_Y[idx]) 
                            if (n_score>0.5) or (p_score>0.5)]
    c_ax.set_title('Dx: '+', '.join(stat_str)+'\nPDx: '+', '.join(pred_str))
    c_ax.axis('off')
fig.savefig('trained_img_predictions.png')